In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

class TELMClassifier:
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        self.input_size = input_size
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.output_size = output_size

        # Xavier initialization for weights
        self.weights1 = np.random.randn(input_size, hidden_size1) / np.sqrt(input_size)
        self.bias1 = np.random.randn(hidden_size1)
        self.weights2 = np.random.randn(hidden_size1, hidden_size2) / np.sqrt(hidden_size1)
        self.bias2 = np.random.randn(hidden_size2)

        # Initialize output weights randomly
        self.output_weights = None

    def train(self, X, y):
        # Compute the output of the first hidden layer
        hidden_output1 = np.dot(X, self.weights1) + self.bias1
        hidden_output1 = np.maximum(hidden_output1, 0)  # ReLU activation

        # Compute the output of the second hidden layer
        hidden_output2 = np.dot(hidden_output1, self.weights2) + self.bias2
        hidden_output2 = np.maximum(hidden_output2, 0)  # ReLU activation

        # Compute output weights using Moore-Penrose pseudoinverse
        self.output_weights = np.linalg.pinv(hidden_output2) @ y

    def predict(self, X):
        # Compute the output of the first hidden layer
        hidden_output1 = np.dot(X, self.weights1) + self.bias1
        hidden_output1 = np.maximum(hidden_output1, 0)  # ReLU activation

        # Compute the output of the second hidden layer
        hidden_output2 = np.dot(hidden_output1, self.weights2) + self.bias2
        hidden_output2 = np.maximum(hidden_output2, 0)  # ReLU activation

        # Compute the output
        output = hidden_output2 @ self.output_weights
        return output

# Load email spam dataset
df = pd.read_csv('mail_data.csv')

# Encode 'spam' as 0 and 'ham' as 1
label_encoder = LabelEncoder()
df['Category'] = label_encoder.fit_transform(df['Category'])

# Split data into features and target
X = df['Message']
y = df['Category']

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(min_df=1, stop_words='english', lowercase=True)
X_features = tfidf_vectorizer.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)

# Initialize and train the TELM classifier
input_size = X_train.shape[1]
hidden_size1 = 100
hidden_size2 = 100
output_size = len(np.unique(y_train))

telm = TELMClassifier(input_size, hidden_size1, hidden_size2, output_size)
telm.train(X_train, y_train)

# Make predictions on the test set
y_pred = telm.predict(X_test)
predicted_labels = np.argmax(y_pred, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print("Accuracy:", accuracy)
